In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('train.csv')
data.head()

,item_id,title,description,price,category_id
0,0,Картина,Гобелен. Размеры 139х84см.,1000.0,19
1,1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22
2,2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37
3,3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43
4,4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1


In [3]:
data.insert(1 , 'text', data.apply(lambda x: x['title'] + ' ' + x['description'], axis=1))
data.drop(['title', 'description'], axis=1, inplace=True)

data.head()

,item_id,text,price,category_id
0,0,Картина Гобелен. Размеры 139х84см.,1000.0,19
1,1,Стулья из прессованной кожи Продам недорого 4 ...,1250.0,22
2,2,"Домашняя мини баня Мини баня МБ-1(мини сауна),...",13000.0,37
3,3,"Эксклюзивная коллекция книг ""Трансаэро"" + пода...",4000.0,43
4,4,Ноутбук aser Продаётся ноутбук ACER e5-511C2TA...,19000.0,1


In [4]:
import re

#Let's use 'GOOD OLD'regular expressions for cleanup and standard func '.lower()' to make words similar
def del_symb(text):
    #return re.sub(r"[^a-zA-Zа-яёА-Я0-9]", " ", text.lower())
    return re.sub(r"[^a-zA-Zа-яёА-Я]", " ", text.lower())

In [5]:
data['text'] = data['text'].apply(del_symb)

data.head()

,item_id,text,price,category_id
0,0,картина гобелен размеры х см,1000.0,19
1,1,стулья из прессованной кожи продам недорого ...,1250.0,22
2,2,домашняя мини баня мини баня мб мини сауна ...,13000.0,37
3,3,эксклюзивная коллекция книг трансаэро пода...,4000.0,43
4,4,ноутбук aser продаётся ноутбук acer e c ta...,19000.0,1


In [6]:
def digits_cleaner(text):
    text_without_dig = [w for w in text.split() if not w.isdigit()] 
    return ' '.join(text_without_dig)

In [7]:
data['text'] = data['text'].apply(digits_cleaner)
data.head()

,item_id,text,price,category_id
0,0,картина гобелен размеры х см,1000.0,19
1,1,стулья из прессованной кожи продам недорого ст...,1250.0,22
2,2,домашняя мини баня мини баня мб мини сауна пре...,13000.0,37
3,3,эксклюзивная коллекция книг трансаэро подарок ...,4000.0,43
4,4,ноутбук aser продаётся ноутбук acer e c ta куп...,19000.0,1


## 2. Предобработка данных

После токенизации нам необходимо привести все слова обращений к некоторой нормальной форме. Такое преобразование включает в себя:
- Приведение слов к нормальной форме с помощью стемминга или лемматизации (будем использовать лемматизацию)
- Осуществление списка заданных замен

In [6]:
#Перед началом преобразований посчитаем количество уникальных слов во всех обращениях:
n_words = len(set(' '.join(data['text']).split()))
n_words

376867

In [7]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import Stemmer as stm


In [8]:
def process(input_text):
    language = 'russian'
    #stemmer = stm.Stemmer(language)
    
    # List of stop words 
    stop_words = stopwords.words(language)
    # Remove the stop words 
    input_text = [x for x in input_text.split() if not x in stop_words]  
    # Stemming on the words 
    #input_text = [stemmer.stemWord(x) for x in input_text]
    
    return ' '.join(input_text)

In [9]:
data['text'] = data['text'].apply(process)

In [10]:
data.head()

,item_id,text,price,category_id
0,0,картина гобелен размеры х см,1000.0,19
1,1,стулья прессованной кожи продам недорого стула...,1250.0,22
2,2,домашняя мини баня мини баня мб мини сауна пре...,13000.0,37
3,3,эксклюзивная коллекция книг трансаэро подарок ...,4000.0,43
4,4,ноутбук aser продаётся ноутбук acer e c ta куп...,19000.0,1


In [11]:
import pymorphy2
from pymorphy2 import MorphAnalyzer

from tqdm import tqdm

In [12]:
def to_lemmatize1(df):
    lemmatizer = MorphAnalyzer()
    lemm_func = lambda text: ' '.join([lemmatizer.normal_forms(word)[0] for word in text.split()])
    df['lemmatized_text'] = df['text'].apply(lemm_func)
    
    return df

In [39]:
def to_lemmatize2(df):
    all_word_str = " ".join(df["text"])
    all_word_list = all_word_str.split()
    all_unique_word = pd.Series(all_word_list).unique()
    lemmatized_word_dict = {}
    lemmatizer = MorphAnalyzer()
    for word in tqdm(all_unique_word):
        #Начало кода
        key = lemmatizer.normal_forms(word)
        lemmatized_word_dict[word] = lemmatizer.normal_forms(word)
        #Конец кода
    lemm_func = lambda text: ' '.join([lemmatized_word_dict[word][0] for word in text.split()])
    df['lemmatized_text'] = df['text'].apply(lemm_func)
    return df, all_unique_word

In [13]:
%%time

data = to_lemmatize1(data)

CPU times: user 31min 40s, sys: 5.08 s, total: 31min 45s
Wall time: 31min 46s


In [14]:
data.to_csv('new_transformed_data.csv')

In [15]:
new_n_words = len(set(" ".join(data["lemmatized_text"]).split()))


In [16]:
print("количество уникальных слов до обработки обращений:", n_words)
print("количество уникальных слов после обработки обращений:", new_n_words)

количество уникальных слов до обработки обращений: 376867
количество уникальных слов после обработки обращений: 211801


In [17]:
data.head()

,item_id,text,price,category_id,lemmatized_text
0,0,картина гобелен размеры х см,1000.0,19,картина гобелен размер х сантиметр
1,1,стулья прессованной кожи продам недорого стула...,1250.0,22,стул прессовать кожа продать недорого стул све...
2,2,домашняя мини баня мини баня мб мини сауна пре...,13000.0,37,домашний минь баня минь баня мб минь сауна пре...
3,3,эксклюзивная коллекция книг трансаэро подарок ...,4000.0,43,эксклюзивный коллекция книга трансаэро подарок...
4,4,ноутбук aser продаётся ноутбук acer e c ta куп...,19000.0,1,ноутбук aser продаваться ноутбук acer e c ta к...


In [20]:
texts = [text.split() for text in data['lemmatized_text']]

In [21]:
from gensim.models import Word2Vec
import tqdm

import numpy as np

/home/dimitry/anaconda3/lib/python3.7/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [22]:
EMBEDDING_DIM = 100
w2v = Word2Vec(sentences=texts, size=EMBEDDING_DIM, window=4, sg=0, iter=5, min_count=1)

In [30]:
w2v.most_similar('струна')

/home/dimitry/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('гитара', 0.6751211881637573),
 ('лада', 0.6661633253097534),
 ('колок', 0.6518442630767822),
 ('фендеровский', 0.6439658403396606),
 ('флойда', 0.6391646862030029),
 ('колка', 0.6253440380096436),
 ('ладовый', 0.6135900020599365),
 ('электрогитара', 0.6088297367095947),
 ('звучка', 0.5997499227523804),
 ('rg421', 0.5995813608169556)]